In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import time

from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model.stochastic_gradient import SGDRegressor
from sklearn.metrics import mean_squared_error

from math import sqrt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
#Importing dataset
df = pd.read_csv("dataset/train.csv")
df_val = pd.read_csv("dataset/train_val1.csv")

In [3]:
print(df.shape)
print(df_val.shape)

(334212, 43)
(62835, 43)


We try to find the best number of features

In [4]:
y = df['NumberOfSales']
y_val = df_val['NumberOfSales']
X = df.drop(['NumberOfSales'], axis=1)
X_val = df_val.drop(['NumberOfSales'], axis=1)

In [5]:
dict_df = {'loss_function' : [], 'alpha' : [], 'epsilon' : [], 'learning_rate' : [], 'var_threshold' : [],'num_feat' : [], 'mse' : []}

In [ ]:
loss_fun = ['squared_loss']
learning_r = ['constant', 'optimal', 'invascaling']

start = time.time()

for l_f in loss_fun:
    for l_r in learning_r:
        for a in np.arange(.0001, .01 , .0001):
            for e in np.arange(.1, .5, .1):
                sgd = SGDRegressor(loss=l_f, alpha=a, epsilon=e, learning_rate=l_r, shuffle=False, random_state=1234567890)
                dict_df['loss_function'].append(l_f)
                dict_df['alpha'].append(a)
                dict_df['epsilon'].append(e)
                dict_df['learning_rate'].append(l_r)
                for v in np.arange(.001, .03, .0005):
                    sfm = SelectFromModel(sgd, threshold=v)
                    sfm.fit(X, y)
                    n_features = sfm.transform(X).shape[1]
                    dict_df['var_threshold'].append(v)
                    dict_df['num_feat'].append(n_features)
                    if n_features == 0:
                        dict_df['mse'].append(99999999999999999)
                    else:
                        mask = sfm.get_support()
                        X_mask = X.loc[:, mask]
                        X_val_mask = X_val.loc[:, mask]
                        sgd.fit(X_mask, y)
                        y_pred=sgd.predict(X_val_mask)
                        dict_df['mse'].append(sqrt(mean_squared_error(y_val,y_pred)))
            
end = time.time() - start
end

In [ ]:
pd.DataFrame(dict_df).to_csv("res_sgd_1.csv")